# chatbot project
GD 노드 12

프로세스는 다음과 같다.
1. 데이터 로드 및 전처리
2. 모델링 (transformer)
3. 모델 훈련
4. 모델 평가

## 1. 데이터 로드 및 전처리

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.2.0


In [2]:
# data load
path_to_file = os.getenv('HOME')+'/aiffel/transformer_chatbot/Chatbot_data-master/ChatbotData .csv'
data = pd.read_csv(path_to_file)

In [3]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
# data check
src = []
tgt = []
for s,t in zip(data['Q'],data['A']):
    src.append(str(s))
    tgt.append(str(t))

for s,t in zip(src[:20],tgt[:20]):
    print(">> ", s, "+",t)


>>  12시 땡! + 하루가 또 가네요.
>>  1지망 학교 떨어졌어 + 위로해 드립니다.
>>  3박4일 놀러가고 싶다 + 여행은 언제나 좋죠.
>>  3박4일 정도 놀러가고 싶다 + 여행은 언제나 좋죠.
>>  PPL 심하네 + 눈살이 찌푸려지죠.
>>  SD카드 망가졌어 + 다시 새로 사는 게 마음 편해요.
>>  SD카드 안돼 + 다시 새로 사는 게 마음 편해요.
>>  SNS 맞팔 왜 안하지ㅠㅠ + 잘 모르고 있을 수도 있어요.
>>  SNS 시간낭비인 거 아는데 매일 하는 중 + 시간을 정하고 해보세요.
>>  SNS 시간낭비인데 자꾸 보게됨 + 시간을 정하고 해보세요.
>>  SNS보면 나만 빼고 다 행복해보여 + 자랑하는 자리니까요.
>>  가끔 궁금해 + 그 사람도 그럴 거예요.
>>  가끔 뭐하는지 궁금해 + 그 사람도 그럴 거예요.
>>  가끔은 혼자인게 좋다 + 혼자를 즐기세요.
>>  가난한 자의 설움 + 돈은 다시 들어올 거예요.
>>  가만 있어도 땀난다 + 땀을 식혀주세요.
>>  가상화폐 쫄딱 망함 + 어서 잊고 새출발 하세요.
>>  가스불 켜고 나갔어 + 빨리 집에 돌아가서 끄고 나오세요.
>>  가스불 켜놓고 나온거 같아 + 빨리 집에 돌아가서 끄고 나오세요.
>>  가스비 너무 많이 나왔다. + 다음 달에는 더 절약해봐요.


### 데이터 전처리
영문자를 소문자로 변환 및 기본 알파벳을 제외한 나머지 제거   
중복 제거 및 토큰화하기 및 사전 구축

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^0-9ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    
    return sentence

In [6]:
# using konlpy
vocab_size = 20000
from konlpy.tag import Mecab
from collections import Counter
tokenizer = Mecab()
max_len = 50

def build_corpus(src, tgt, l, num_words=vocab_size, dup=0):
    if dup == 0:
        sen_idx = {}
        src_u = []
        tgt_u = []

        for sen1,sen2 in zip(src,tgt):
            if sen1 not in sen_idx:
                sen_idx[sen1] = 1
                src_u.append(sen1)
                tgt_u.append(sen2)

        sen_idx = {}
        src = []
        tgt = []

        for sen1,sen2 in zip(src_u,tgt_u):
            if sen2 not in sen_idx:
                sen_idx[sen2] = 1
                src.append(sen1)
                tgt.append(sen2)

    
    
    src_p = []
    tgt_p = []
    for s,t in zip(src,tgt):
        src_p.append(preprocess_sentence(s))
        tgt_p.append(preprocess_sentence(t))
    
    src_tok = []
    tgt_tok = []
    word_tok = []
    
    for s in src_p:
        tmp = tokenizer.morphs(s)
        src_tok.append(tmp)
        word_tok.append(tmp)
    
    for t in tgt_p:
        tmp = tokenizer.morphs(t)
        tgt_tok.append(tmp)
        word_tok.append(tmp)
        
    words = np.concatenate(word_tok).tolist()
    counter = Counter(words)
    counter = counter.most_common(num_words-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<EOS>'] + [key for key, _ in counter]
    # 사전 구성
    word_to_index = {word:index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
    # 변환 text to index
    src_data = list(map(wordlist_to_indexlist, src_tok))
    tgt_data = list(map(wordlist_to_indexlist, tgt_tok))
    
    src_l = []
    tgt_l = []
    
    for s,t in zip(src_data,tgt_data):
        if len(s) <= l and len(t) <= l:
            src_l.append(s)
            tgt_l.append(t)
            
    
    
    return src_l, tgt_l, word_to_index

que_corpus, ans_corpus, word_to_index = build_corpus(src,tgt,max_len)
for q,a in zip(que_corpus[:20], ans_corpus[:20]):
    print(q, a)

[2057, 209, 2581, 105] [264, 9, 136, 9, 39, 4]
[284, 3559, 599, 1043, 13] [624, 17, 1492, 4]
[279, 2058, 480, 65, 271, 261, 9, 11, 44, 35] [275, 16, 672, 10, 34, 4]
[3560, 1126, 36] [5180, 5, 5181, 19, 34, 4]
[3561, 1535, 3562, 13] [121, 2000, 188, 6, 24, 51, 5182, 4]
[544, 190, 1044, 132, 42, 7, 19, 627] [57, 116, 11, 15, 8, 38, 23, 15, 30, 4]
[544, 73, 1536, 102, 14, 21, 46, 412, 7, 6, 146] [73, 8, 2452, 11, 17, 18, 12, 4]
[544, 18, 37, 22, 66, 1127, 11, 35, 152, 17, 521] [2864, 7, 6, 846, 254, 4]
[494, 318, 17] [90, 27, 23, 217, 14, 28, 4]
[494, 16, 180, 102, 24, 10, 35] [180, 31, 969, 12, 4]
[3563, 33, 206, 40, 3564] [266, 16, 121, 3278, 14, 28, 4]
[3565, 15, 239, 2059, 1757] [2059, 8, 3279, 56, 12, 4]
[3566, 3567, 3568, 2582] [291, 123, 11, 513, 1162, 7, 12, 4]
[2060, 1045, 1758, 11, 1537, 13] [523, 228, 26, 844, 143, 982, 11, 562, 12, 4]
[2060, 325, 54, 78, 825, 35, 4] [424, 140, 26, 6, 67, 1873, 17, 49, 4]
[2060, 325, 3569, 863, 1128, 25, 13] [608, 7, 24, 707, 105]
[454, 436, 33

In [7]:
index_to_word = {index:word for word, index in word_to_index.items()}

### Augmentation
wiki data를 이용하여 훈련시킨 word2vec을 이용하여 데이터를 augmentation한다.   
총 데이터가 30000개 정도가 되도록 한다.   

In [8]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load("w2v_wiki.wv", mmap='r')

/home/aiffel-dj28/anaconda3/envs/aiffel/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
def lexical_sub(sentence, word2vec, top=0):
    import random

    res = ""
    toks = sentence.split()
    
    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[top][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [10]:
# augmentation
from tqdm import tqdm_notebook

src_corpus = []
tgt_corpus = []
for idx in tqdm_notebook(range(7731)):
    
    old_src = []
    for w in que_corpus[idx]:
        ow = index_to_word[w]
        old_src.append(ow)
    old_src = ' '.join(old_src)

    old_tgt = []
    for w in ans_corpus[idx]:
        ow = index_to_word[w]
        old_tgt.append(ow)
    old_tgt = ' '.join(old_tgt)

    
    new_src = [None]*3
    new_tgt = [None]*3
    
    new_src[0] = old_src
    new_tgt[0] = old_tgt
    
    new_src[1] = lexical_sub(old_src, wv)
    new_src[2] = lexical_sub(old_src, wv, 1)
#    new_src[3] = lexical_sub(old_src, wv, 2)
#    new_src[4] = lexical_sub(old_src, wv, 3)
    
    new_tgt[1] = lexical_sub(old_tgt, wv)
    new_tgt[2] = lexical_sub(old_tgt, wv, 1)
#    new_tgt[3] = lexical_sub(old_tgt, wv, 2)
#    new_tgt[4] = lexical_sub(old_tgt, wv, 3)
    
    
    for i in new_src:
        for j in new_tgt:
            if i is not None and j is not None:
                src_corpus.append(i)
                tgt_corpus.append(j)
    
    
#     if new_src_1 is not None and new_tgt_1 is not None:
#         src_corpus.append(new_src_1)
#         tgt_corpus.append(new_tgt_1)
        
#     if new_src_2 is not None and new_tgt_2 is not None:
#         src_corpus.append(new_src_2)
#         tgt_corpus.append(new_tgt_2)
        
#     if new_src_3 is not None and new_tgt_3 is not None:
#         src_corpus.append(new_src_3)
#         tgt_corpus.append(new_tgt_3)
        
#     if new_src_4 is not None and new_tgt_4 is not None:
#         src_corpus.append(new_src_4)
#         tgt_corpus.append(new_tgt_4)
    
print(src_corpus[:20])
print(src[:20])
print(len(src_corpus))

/home/aiffel-dj28/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/7731 [00:00<?, ?it/s]

['12 시 땡 !', '12 시 땡 !', '12 시 땡 !', '12 시 고광렬 ! ', '12 시 고광렬 ! ', '12 시 고광렬 ! ', '12 시 우스개 ! ', '12 시 우스개 ! ', '12 시 우스개 ! ', '1 지망 학교 떨어졌 어', '1 장래희망 학교 떨어졌 어 ', '3 박 4 일 놀 러 가 고 싶 다', '3 박 4 일 놀 러 가 고 싶 다', '3 박 4 일 놀 러 가 고 싶 다', '3 박 4 일 지스 러 가 고 싶 다 ', '3 박 4 일 지스 러 가 고 싶 다 ', '3 박 4 일 지스 러 가 고 싶 다 ', '3 박 4 일 놀 러 고레 고 싶 다 ', '3 박 4 일 놀 러 고레 고 싶 다 ', '3 박 4 일 놀 러 고레 고 싶 다 ']
['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네', 'SD카드 망가졌어', 'SD카드 안돼', 'SNS 맞팔 왜 안하지ㅠㅠ', 'SNS 시간낭비인 거 아는데 매일 하는 중', 'SNS 시간낭비인데 자꾸 보게됨', 'SNS보면 나만 빼고 다 행복해보여', '가끔 궁금해', '가끔 뭐하는지 궁금해', '가끔은 혼자인게 좋다', '가난한 자의 설움', '가만 있어도 땀난다', '가상화폐 쫄딱 망함', '가스불 켜고 나갔어', '가스불 켜놓고 나온거 같아', '가스비 너무 많이 나왔다.']
37636


### 전처리
augmentation한 데이터에 대하여 전처리 후 벡터화하는 과정과 데이터 셋 분리까지 한다.

In [11]:
new_que_corpus, new_ans_corpus, word_to_index = build_corpus(src_corpus,tgt_corpus,max_len,dup=1)

In [12]:
len(new_que_corpus)

37636

In [13]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [14]:
ans = []
for a in new_ans_corpus:
    ans.append([word_to_index["<BOS>"]] + a + [word_to_index["<EOS>"]])

In [15]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(new_que_corpus, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01)

print("enc_train :", len(enc_train), "enc_val :", len(enc_val))
print("dec_train :", len(dec_train), "dec_val :",len(dec_val))

enc_train : 37259 enc_val : 377
dec_train : 37259 dec_val : 377


In [16]:
len(enc_train[0])

32

In [17]:
len(dec_train[0])

42

## 2. 모델링 (transformer)

In [18]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table
print("슝=3")

슝=3


In [19]:
# Mask  생성하기
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask
print("슝=3")

슝=3


In [20]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights
print("슝=3")

슝=3


In [21]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out
print("슝=3")

슝=3


In [22]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn
print("슝=3")

슝=3


In [23]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("슝=3")

슝=3


In [24]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns
print("슝=3")

슝=3


In [25]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [26]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

print("슝=3")

슝=3


## 3. 모델 훈련
transformer 모델을 훈련시킨다.   
하이퍼 파라미터는 아래와 같다.   
n_layers=2   
d_model=128   
n_heads=8   
d_ff=256   

In [27]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=2,
    d_model=128,
    n_heads=8,
    d_ff=256,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=42,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 128
print("슝=3")

슝=3


In [28]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

print("슝=3")

슝=3


In [29]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)
print("슝=3")

슝=3


In [30]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

print("슝=3")

슝=3


In [31]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

print("슝=3")

슝=3


In [56]:
# 훈련시키기
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 5

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/home/aiffel-dj28/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

## 4. 모델 평가
챗봇 모델을 평가하기 위하여 bleu 점수를 이용한다.

In [57]:
# !pip install nltk # nltk가 설치되어 있지 않은 경우 주석 해제
from nltk.translate.bleu_score import sentence_bleu

reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다']
번역문: ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score: 8.190757052088229e-155


In [58]:
print("1-gram:", sentence_bleu([reference], candidate, weights=[1, 0, 0, 0]))
print("2-gram:", sentence_bleu([reference], candidate, weights=[0, 1, 0, 0]))
print("3-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 1, 0]))
print("4-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 0, 1]))

1-gram: 0.5
2-gram: 0.18181818181818182
3-gram: 2.2250738585072626e-308
4-gram: 2.2250738585072626e-308


In [59]:
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.5
BLEU-2: 0.18181818181818182
BLEU-3: 0.010000000000000004
BLEU-4: 0.011111111111111112

BLEU-Total: 0.05637560315259291


### bleu 평가 및 결과
위에서 나눈 데이터를 이용하여 bleu 점수를 계산한다.(0.55 정도)   
그리고 예제에 대해서 결과도 확인한다.   

In [60]:
# translate()

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)
    
    print(sentence)
    s = src_tokenizer.morphs(sentence)
    tokens = []
    for i in s:
        tokens.append(word_to_index[i])
    pieces = tokens        
        
#     pieces = src_tokenizer.encode_as_pieces(sentence)
#     tokens = src_tokenizer.encode_as_ids(sentence)
        
    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([1], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if predicted_id == 3:
            #result = tgt_tokenizer.decode_ids(ids)
            result = []
            for i in ids:
                result.append(index_to_word[i])
            print(result)
            
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    #result = tgt_tokenizer.decode_ids(ids)
    result = ''
    for i in ids:
        result += index_to_word[i]
    
    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model, src_tokenizer, tgt_tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

    return result
print("슝=3")

슝=3


In [61]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]

#         src_sentence = tokenizer.decode_ids((src_tokens.tolist()))
#         tgt_sentence = tokenizer.decode_ids((tgt_tokens.tolist()))

        src_sentence = []
        tgt_sentence = []
        
        for w in src_tokens:
            if w != 0:
                ow = index_to_word[w]
                src_sentence.append(ow)

        src_sentence = ' '.join(src_sentence)
        
        for w in tgt_tokens:
            if w != 0 and w != 1 and w != 3:
                ow = index_to_word[w]
                tgt_sentence.append(ow)
   
        tgt_sentence = ' '.join(tgt_sentence)
        
        reference = preprocess_sentence(tgt_sentence).split()
        candidate = translate(src_sentence, transformer, tokenizer, tokenizer)
        
        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)
print("슝=3")

슝=3


In [62]:
eval_bleu(enc_val[:3], dec_val[:3], True)

/home/aiffel-dj28/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3 [00:00<?, ?it/s]

돈 어디 갔 지
['퍼', '가', '쇼키']
Source Sentence:  돈 어디 갔 지
Model Prediction:  ['퍼', '가', '쇼키']
Real:  ['퍼', '가', '쇼키']
Score: 0.562341

이별 준비 하 려고
['쉽', '지', '않', '은', '마음', '고생', '많', '았', '어요', '.']
Source Sentence:  이별 준비 하 려고
Model Prediction:  ['쉽', '지', '않', '은', '마음', '고생', '많', '았', '어요', '.']
Real:  ['쉽', '지', '않', '은', '결정', '이', '었', '을', '텐데', '마음', '고생', '많', '았', '어요', '.']
Score: 0.476508

행복 한 돌 싱 슬하
['매일', '매일', '행복', '한', '일', '이', '생기', '시', '로', '나', '봅니다', '.']
Source Sentence:  행복 한 돌 싱 슬하
Model Prediction:  ['매일', '매일', '행복', '한', '일', '이', '생기', '시', '로', '나', '봅니다', '.']
Real:  ['매일', '매일', '행복', '한', '일', '이', '생기', '시', '가', '길', '바랄', '게요']
Score: 0.613230

Num of Sample: 3
Total Score: 0.5506931086532787


In [63]:
eval_bleu(enc_val[::10], dec_val[::10], verbose=False)

/home/aiffel-dj28/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/38 [00:00<?, ?it/s]

돈 어디 갔 지
['퍼', '가', '쇼키']
재미 없 어
['변화', '쓰루타', '필요', '해보여요', '.']
짝 녀 생각 에 잠 을 못 자 겠 어 .
['짝사랑', '이', '자기만족', '을', '복잡', '하', '게', '만들', '어요', '.']
짝 녀랑 카톡 했 던 거 다시 읽 어 보 는데 매우 다 만 물 어 보 네 .
['상대', '분', '이', '아직', '관심', '이', '덜한', '걸', '수', '도', '있', '어요', '.']
새벽 노승호
['조용', '하', '니', '혼자', '생각', '하', '기', '좋', '죠', '.']
술기운 에 연락 했 는데 .
['후회', '하', '지', '않', '길', '바라', '아', '리페', '.']
3 년 부부 돈 싸움 . 헤어짐
['돈', '것', '문제', '죠', '.']
소개팅 응우옌 반 쑤 언 고 싶 다
['친구', '한테', '부탁', '해', '보', '세요', '.']
내 가 힘든 불구
['자신', '이', '알', '고', '있', '겠', '죠', '.']
손발 이 차가워
['지압', '키', '리바', '운동', '해', '보', '세요', '.']
봄 에 동물원 쓰루 타 는 거 어때 ?
['동심', '으로', '돌아갈', '거', '같', '아요', '.']
좋 아 하 는 여자 드버 생겼 는데 떨려 죽 겠 음 .
['설렘', '을', '감추', '는', '게', '더', '힘들', '때', '네요', '.']
겨울 에 는 온천 이 지 !
['몸', '은', '뜨겁', '고', '머리', '는', '차갑', '게', '!']
더 이상 나 한테 안 왔 으면 좋 겠 어
['명확', '하', '게', '거부', '의', '의사', '를', '표현', '하', '세요', '.']
오늘 그 사람 생일 이 네
['생각날', '수', '밖', '에', '없', '다고', '생각', '해요', '.']
썸 쿠 는데 머리 아파 신경 쓰여 .
['하루', '먹', '고',

In [64]:
translate("지루하다, 놀러가고 싶어.", transformer, tokenizer, tokenizer)

지루하다 , 놀러가고 싶어 .
['제', '생각', '해', '보', '세요', '.']


['제', '생각', '해', '보', '세요', '.']

In [65]:
translate("오늘 일찍 일어났더니 피곤하다.", transformer, tokenizer, tokenizer)

오늘 일찍 일어났더니 피곤하다 .
['이제', '이나고', '이', '라도', '괜찮', '아', '했', '나', '봐요', '.']


['이제', '이나고', '이', '라도', '괜찮', '아', '했', '나', '봐요', '.']

In [66]:
translate("간만에 여자친구랑 데이트 하기로 했어.", transformer, tokenizer, tokenizer)

간만에 여자친구랑 데이트 하기로 했어 .
['시간', '도', '흐르', '고', '많', '았', '죠', '.']


['시간', '도', '흐르', '고', '많', '았', '죠', '.']

In [67]:
translate("집에 있는다는 소리야.", transformer, tokenizer, tokenizer)

집에 있는다는 소리야 .
['조금', '씩', '조금', '씩', '정리', '해', '보', '세요', '.']


['조금', '씩', '조금', '씩', '정리', '해', '보', '세요', '.']

## 회고 및 루브릭 평가

### 루브릭 평가 항목
1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?
	(챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.)
2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?
	(과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.)
3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?
	(주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.)

### 평가 항목에 대한 수행
1. 훈련데이터 augmentation과 전처리 과정을 진행하였다.
2. transformer 모델을 안정적으로 훈련시켰다.
3. 예문을 포함하여 그럴듯한 형태로 답을 하는 것을 확인하였다. 

### 회고

노션에 공지된 꼭 포함이 되어야 할 점
- 이번 프로젝트에서 **어려웠던 점,**
- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.
- **자기 다짐**

---
- **어려웠던 점**    
word2vec 모델이 로드가 되지 않아서 데이터를 이용하여 처음부터 학습시켜서 이용을 했다.

---
- **알아낸 점**    
wiki 데이터를 이용하는 방법을 알 수 있었고 bleu 점수에 대해서 알 수 있었다.

- **모호한 점**    
챗봇 점수와 실제적으로 느껴지는 성능 사이에 괴리감이 있다.

---
- **시도한 것들**   
wiki 데이터를 이용하여 임베딩 모델을 만들고 실험하였다.

---
- **루브릭 평가 관련 지표**   
제 예상에는 모두 달성되었다고 생각한다. 그 이유는 위에 있는 **평가 항목에 대한 수행**에 나와있다.  
- **자기 다짐** 및 **나의 생각들**      
이전 노드에 이어서 transformer를 이용한다. NLP에서 데이터를 augmentation에 대해서 생각해 볼 수 있는 시간이였다. 

### word2vec 훈련
wiki data를 이용하여 w2v을 만든다.    
이때 wikiextractor를 이용하여 데이터를 얻고 훈련한다.

In [ ]:
import os
path = os.getenv('HOME')+'/w2v/wikiextractor/wikiextractor/text/wiki_sdata.txt'

f = open(path,encoding="utf8")
i=0
while True:
    line = f.readline()
    if line != '\n':
        i=i+1
        print("%d번째 줄 :"%i + line)
    if i==5:
        break 
f.close()

In [ ]:
from konlpy.tag import Okt  
okt=Okt()
fread = open(path, encoding="utf8")
# 파일을 다시 처음부터 읽음.
n=0
result = []

while True:
    line = fread.readline() #한 줄씩 읽음.
    if not line: break # 모두 읽으면 while문 종료.
    n=n+1
    if n%5000==0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 While문."%n)
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp) # 결과에 저장
fread.close()

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, vector_size=300, window=5, min_count=5, workers=4, sg=1)

In [ ]:
w2v = model.wv

In [ ]:
w2v.save("w2v_wiki.wv")

In [ ]:
model.wv.most_similar('사람')